<a href="https://colab.research.google.com/github/Jaimemorillo/ShouldIwatchThisMovie/blob/master/memoria_encoding_textos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('default')

### Bibliografy
- https://towardsdatascience.com/text-encoding-a-review-7c929514cccf

- https://realpython.com/python-keras-text-classification/

# Sentences

In [ ]:
corpus = [
    'Este es el primer documento.',
    'Este documento es el segundo documento.',
    'Este es el tercero.',
    '¿Es este el primer documento?'
    ]

## Bag of words (not ordered)
Sentence as vector


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [ ]:
vectorizer.get_feature_names()

['documento', 'el', 'es', 'este', 'primer', 'segundo', 'tercero']

In [ ]:
X.toarray()

array([[1, 1, 1, 1, 1, 0, 0],
       [2, 1, 1, 1, 0, 1, 0],
       [0, 1, 1, 1, 0, 0, 1],
       [1, 1, 1, 1, 1, 0, 0]])

In [ ]:
pd.DataFrame(columns=vectorizer.get_feature_names(), data=X.toarray(), index=corpus)

,documento,el,es,este,primer,segundo,tercero
Este es el primer documento.,1,1,1,1,1,0,0
Este documento es el segundo documento.,2,1,1,1,0,1,0
Este es el tercero.,0,1,1,1,0,0,1
¿Es este el primer documento?,1,1,1,1,1,0,0


## One-Hot Encoding (ordered)
Word as vector

In [ ]:
words = vectorizer.get_feature_names()
words

['documento', 'el', 'es', 'este', 'primer', 'segundo', 'tercero']

In [ ]:
word_to_vector = {
    'documento' : [1, 0, 0, 0, 0, 0, 0],
    'el' : [0, 1, 0, 0, 0, 0, 0],
    'es' : [0, 0, 1, 0, 0, 0, 0],
    'este' : [0, 0, 0, 1, 0, 0, 0],
    'primer' : [0, 0, 0, 0, 1, 0, 0],
    'segundo' : [0, 0, 0, 0, 0, 1, 0],
    'tercero' : [0, 0, 0, 0, 0, 0, 1],
}

In [ ]:
corpus_as_array = [x.replace(".", "").replace("¿","").replace("?","").lower().split(" ") for x in corpus]

In [ ]:
corpus_as_array

[['este', 'es', 'el', 'primer', 'documento'],
 ['este', 'documento', 'es', 'el', 'segundo', 'documento'],
 ['este', 'es', 'el', 'tercero'],
 ['es', 'este', 'el', 'primer', 'documento']]

In [ ]:
#@title
one_hot_corpus = np.array([np.array([word_to_vector[x] for x in corpus_as_arrays[i]]) for i in range(0, len(corpus_as_arrays))])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


A cada palabra se le asigna un vector, la manera más sencilla es asignarle un vector que contenga 0's en todas las posiciones menos un 1 en la posición que corresponde con su índice en el diccionario.

In [ ]:
#@title
df_word_to_vector = pd.DataFrame(pd.Series(vectorizer.vocabulary_).sort_values(), columns=['indice'])
df_word_to_vector['vector'] = pd.Series(word_to_vector)
df_word_to_vector

,indice,vector
documento,0,"[1, 0, 0, 0, 0, 0, 0]"
el,1,"[0, 1, 0, 0, 0, 0, 0]"
es,2,"[0, 0, 1, 0, 0, 0, 0]"
este,3,"[0, 0, 0, 1, 0, 0, 0]"
primer,4,"[0, 0, 0, 0, 1, 0, 0]"
segundo,5,"[0, 0, 0, 0, 0, 1, 0]"
tercero,6,"[0, 0, 0, 0, 0, 0, 1]"


Hay que aplicarle la codificación anterior a cada una de las oraciones. Primero necesitamos llevar la frase a formato array separando cada una de las palabras. Y ya podemos mapear cada una de las palabras a su vector correspondiente.

Este es el resultado:

In [ ]:
#@title
data = {
    'sentence_as_array': corpus_as_array,
    'one_hot_sentence': one_hot_corpus
}

df_one_hot = pd.DataFrame(data=data, index=corpus)
df_one_hot

,sentence_as_array,one_hot_sentence
Este es el primer documento.,"[este, es, el, primer, documento]","[[0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0],..."
Este documento es el segundo documento.,"[este, documento, es, el, segundo, documento]","[[0, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0],..."
Este es el tercero.,"[este, es, el, tercero]","[[0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0],..."
¿Es este el primer documento?,"[es, este, el, primer, documento]","[[0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0],..."


Lo que obtenemos ahora es una matriz de cada una de las sentencias, en la cual estamos teniendo en cuenta el orden en el que aparecen las palabras dentro de la misma.

Frase 1:

In [ ]:
#@title
pd.DataFrame(index=df_one_hot.loc[corpus[0],'sentence_as_array'],
             data=df_one_hot.loc[corpus[0],'one_hot_sentence'],
             columns=words)

,documento,el,es,este,primer,segundo,tercero
este,0,0,0,1,0,0,0
es,0,0,1,0,0,0,0
el,0,1,0,0,0,0,0
primer,0,0,0,0,1,0,0
documento,1,0,0,0,0,0,0


Frase 2:

In [ ]:
#@title
pd.DataFrame(index=df_one_hot.loc[corpus[1],'sentence_as_array'],
             data=df_one_hot.loc[corpus[1],'one_hot_sentence'],
             columns=words)

,documento,el,es,este,primer,segundo,tercero
este,0,0,0,1,0,0,0
documento,1,0,0,0,0,0,0
es,0,0,1,0,0,0,0
el,0,1,0,0,0,0,0
segundo,0,0,0,0,0,1,0
documento,1,0,0,0,0,0,0


Para asegurar que las matrices asociadas a cada frase tienen el mismo número de filas, habría que rellenar con un placeholder (vector con todo 0's) aquellas frases de menor longitud tantas filas cómo sea la longitud de la frase más larga.
Para el caso anterior quedaría así:


In [ ]:
placeholder = pd.DataFrame({'*': [0, 0, 0, 0, 0, 0, 0]}, index=words).T

In [ ]:
#@title
pd.DataFrame(index=df_one_hot.loc[corpus[0],'sentence_as_array'],
             data=df_one_hot.loc[corpus[0],'one_hot_sentence'],
             columns=words).append(placeholder)

,documento,el,es,este,primer,segundo,tercero
este,0,0,0,1,0,0,0
es,0,0,1,0,0,0,0
el,0,1,0,0,0,0,0
primer,0,0,0,0,1,0,0
documento,1,0,0,0,0,0,0
*,0,0,0,0,0,0,0


## Index based encoding

In [ ]:
Esta es quizás la manera más intuitiva de codificar una sentencia teniendo en cuenta el orden de las palabras.
Basta con asignar la palabra a su indice en el diccionario.

In [ ]:
pd.DataFrame(pd.Series(vectorizer.vocabulary_).sort_values(), columns=['indice'])

,indice
documento,0
el,1
es,2
este,3
primer,4
segundo,5
tercero,6


Quedaría así:


In [ ]:
index_based_corpus = np.array([np.array([vectorizer.vocabulary_[x] for x in corpus_as_arrays[i]]) for i in range(0, len(corpus_as_arrays))])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [ ]:
#@title
data = {
    'sentence_as_array': corpus_as_array,
    'index_based_sentence': index_based_corpus
}

pd.DataFrame(data, index=corpus)

,sentence_as_array,index_based_sentence
Este es el primer documento.,"[este, es, el, primer, documento]","[3, 2, 1, 4, 0]"
Este documento es el segundo documento.,"[este, documento, es, el, segundo, documento]","[3, 0, 2, 1, 5, 0]"
Este es el tercero.,"[este, es, el, tercero]","[3, 2, 1, 6]"
¿Es este el primer documento?,"[es, este, el, primer, documento]","[2, 3, 1, 4, 0]"


Para asegurar que los vectores tengan el mismo tamaño se suele reservar el indice 0 del vocabulario (empezando este en 1) para rellenar el vector y completar hasta que todos los vectores sean de igual longitud.

El problema de esta codificación es que introduce una distancia numérica entre los textos que realmente no existe. Y por lo tanto no es muy recomendable su utilización.

## Word Embeddings

Word embeddings son un conjunto de técnicas de procesamiento del lenguaje natural que lo que permiten es mapear el significado semántico de las palabras en un espacio geométrico.